In [1]:
from ultralytics import YOLO
model=YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:21<00:00, 2.42MB/s]


In [ ]:
model.train(data="/home/rahul/Desktop/Document-and-Record-Management/YOLOv8/data/data.yaml",epochs=80)

In [ ]:
# yolo = YOLO("/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/pretrained models/best.pt"); yolo.predict(source="/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/license_94.png", save=True)


In [ ]:
model = YOLO("/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/pretrained models/best.pt")
results = model('/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg')

# Extract the coordinates, class IDs, and confidence scores
for result in results:
    boxes = result.boxes.xyxy  # x1, y1, x2, y2 (top-left and bottom-right corner coordinates)
    scores = result.boxes.conf  # Confidence scores
    class_ids = result.boxes.cls  # Class IDs

    for box, score, class_id in zip(boxes, scores, class_ids):
        x1, y1, x2, y2 = box.tolist()  # Convert to list to get the values
        print(f"Class ID: {class_id}, Confidence: {score}, Coordinates: ({x1}, {y1}), ({x2}, {y2})")

In [10]:
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR
import os
import numpy as np

# Define the paths
image_path = '/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg'
model_path = "/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/pretrained models/best.pt"

# Load the trained YOLO model
model = YOLO(model_path)

# Perform inference on an image
results = model(image_path)

# Load the image with OpenCV
image = cv2.imread(image_path)

# Initialize PaddleOCR reader (English language model)
ocr = PaddleOCR(use_angle_cls=True, lang='en') 

# Dictionary to map class IDs to class names
class_map = {
    0: 'address',
    1: 'blood_group',
    2: 'citizenship_number',
    3: 'contact_number',
    4: 'dob',
    5: 'document_type',
    6: 'father_name',
    7: 'license_category',
    8: 'license_number',
    9: 'name'
}

# Generate a color map for classes
def get_color(class_id):
    np.random.seed(class_id)  # Seed for reproducibility
    return tuple(np.random.randint(0, 255, 3).tolist())

# Extract the base name of the image file (without extension)
base_name = os.path.splitext(os.path.basename(image_path))[0]

# Directory to save OCR results
output_dir = 'OCR_Outputs'
os.makedirs(output_dir, exist_ok=True)

# File to save OCR results with the same name as the image
output_file_path = os.path.join(output_dir, f'{base_name}_results.txt')

# Open the file in write mode
with open(output_file_path, 'w') as file:
    # Iterate over each result
    for result in results:
        boxes = result.boxes.xyxy  # Bounding box coordinates
        class_ids = result.boxes.cls  # Class IDs

        # Iterate over each detected box and class ID
        for box, class_id in zip(boxes, class_ids):
            x1, y1, x2, y2 = map(int, box.tolist())  # Convert coordinates to integers
            
            # Crop the image based on the coordinates
            cropped_img = image[y1:y2, x1:x2]

            # Use PaddleOCR to read the text from the cropped image
            ocr_result = ocr.ocr(cropped_img)

            # Extract and save the text recognized by PaddleOCR
            for line in ocr_result:
                for text in line:
                    file.write(f"Class: {class_map.get(int(class_id), 'Unknown')}, Text: {text[1][0]}\n")
                    
            # Draw bounding box and class label on the original image
            color = get_color(int(class_id))
            label = class_map.get(int(class_id), 'Unknown')
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Save the annotated image to the same folder
annotated_image_path = os.path.join(output_dir, f'{base_name}_annotated.jpg')
cv2.imwrite(annotated_image_path, image)



image 1/1 /mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg: 480x640 1 address, 1 blood_group, 1 citizenship_number, 1 contact_number, 1 dob, 1 document_type, 1 father_name, 1 license_category, 1 license_number, 2 names, 93.3ms
Speed: 3.6ms preprocess, 93.3ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)
[2024/08/10 22:58:16] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/rahul/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_t

True